In [1]:
import os
# os.chdir(r'C:/Users/hakan/Jupyter Notebook/notebook/AIFarming/DataAnalysis/')
# os.getcwd()


In [2]:
import pandas as pd
import numpy as np

data_path ='C:/Users/hakan/Jupyter Notebook/data/porto-seguro-safe-driver-prediction/'
train = pd.read_csv(data_path+'train.csv',index_col='id')
test = pd.read_csv(data_path+'test.csv',index_col='id')
submission = pd.read_csv(data_path+'sample_submission.csv',index_col='id')

In [66]:
def resumatable(df):
    print(f'DataFrame Shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['Dtype'])
    summary['Missing'] = (df == -1).sum().values
    summary['Unique'] = df.nunique().values
    summary['Dtype'] = None
    for col in df.columns:
        if 'bin' in col or col =='target':
            summary.loc[col,'Dtype'] = 'Binary'
        elif 'cat' in col:
            summary.loc[col,'Dtype'] = 'Categorical'
        elif df[col].dtype == 'float64':
            summary.loc[col,'Dtype'] = 'Continuous'
        elif df[col].dtype == 'int64':
            summary.loc[col,'Dtype'] = 'Ordinal'
    return summary

In [4]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

#feature engineering

In [5]:
import pandas as pd
data_path ='C:/Users/hakan/Jupyter Notebook/data/porto-seguro-safe-driver-prediction/'
train = pd.read_csv(data_path+'train.csv',index_col='id')
test = pd.read_csv(data_path+'test.csv',index_col='id')
submission = pd.read_csv(data_path+'sample_submission.csv',index_col='id')

In [6]:
all_data = pd.concat([train,test],ignore_index=True)
all_data = all_data.drop('target',axis=1)

In [7]:
all_features = all_data.columns
all_features

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

In [8]:
from sklearn.preprocessing import OneHotEncoder

#one hot encoding for categorical datas
cat_features = [feature for feature in all_features if 'cat' in feature]
onehot_encoder = OneHotEncoder()

encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

In [38]:
drop_features = ['ps_ind_10_bin','ps_ind_11_bin','ps_ind_12_bin','ps_ind_13_bin','ps_ind_14','ps_car_14']

In [32]:
remaining_features = [feature for feature in all_features if ('cat' not in feature and 'calc' not in feature and feature not in drop_features)]

In [33]:
remaining_features

['ps_ind_01',
 'ps_ind_03',
 'ps_ind_06_bin',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_09_bin',
 'ps_ind_15',
 'ps_ind_16_bin',
 'ps_ind_17_bin',
 'ps_ind_18_bin',
 'ps_reg_01',
 'ps_reg_02',
 'ps_reg_03',
 'ps_car_11',
 'ps_car_12',
 'ps_car_13',
 'ps_car_15']

Sparse한 (ex : Onehot encoded) matrix를 다룰 때 scipy.sparse를 사용한다. 대규모 행렬을 다룰 때 메모리 문제를 해결하기 위해 사용한다. scipy.sparse의 matrix를 만드는 방법은 sparse matrix 구성요소를 직접 입력하는 방법과 numpy.ndarray를 입력하는 방법이 있다.

In [12]:
from scipy import  sparse
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_features]),encoded_cat_matrix],format='csr')

In [13]:
all_data_sprs.shape, train.shape, test.shape

((1488028, 201), (595212, 58), (892816, 57))

In [14]:
num_train = len(train)
print(num_train)

X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

595212


In [15]:
import numpy as np
def eval_gini(y_true, y_pred):
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0]
    L_mid = np.linspace(1/n_samples, 1, n_samples)

    pred_order = y_true[y_pred.argsort()]
    true_order = y_true[y_true.argsort()]
    
    L_pred = np.cumsum(pred_order)/np.sum(true_order)
    G_pred = np.sum(L_mid - L_pred)
    
    
    L_true = np.cumsum(true_order)/np.sum(true_order)
    G_true = np.sum(L_mid - L_true)

    #Normalized Gini coefficient
    return G_pred/G_true    

In [16]:
#gini for LGBM
def gini(preds,dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels,preds), True

In [17]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

In [18]:
print(folds.split(X,y))

<generator object _BaseKFold.split at 0x000001C22A04A350>


In [19]:
lgb_params = {
    'objective':'binary',
    'learning_rate':0.01,
    'force_row_wise':True,
    'random_state':0,
    'scale_pos_weight': 2,
}

In [20]:
oof_val_preds = np.zeros(X.shape[0])
oof_test_preds = np.zeros(X_test.shape[0])

In [21]:
import lightgbm as lgb
#Normal Test
for idx, (train_idx,valid_idx) in enumerate(folds.split(X,y)):
    print('#'*40,f'폴드 {idx+1} / {folds.n_splits}','#'*40)
    #Train, Validation data
    X_train, y_train = X[train_idx],y[train_idx]
    X_valid, y_valid = X[valid_idx],y[valid_idx]

    #data set for lgbm
    lgb_train = lgb.Dataset(X_train,y_train)
    lgb_valid = lgb.Dataset(X_valid,y_valid)

    # train lgbm model
    lgb_model = lgb.train(params=lgb_params,
                         train_set = lgb_train,
                         num_boost_round=1000,
                         valid_sets=lgb_valid,
                         feval = gini,                                                   
                         callbacks=[lgb.early_stopping(stopping_rounds=5)])

    #oof prediction based on test data
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    #based in validation data
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)

    #gini
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'fold {idx+1} gini score : {gini_score}\n')

######################################## 폴드 1 / 5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[24]	valid_0's binary_logloss: 0.155301	valid_0's gini: 0.243445
fold 1 gini score : 0.2434451362266238

######################################## 폴드 2 / 5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1093
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[L

In [22]:
#is_unbalance
for idx, (train_idx,valid_idx) in enumerate(folds.split(X,y)):
    print('#'*40,f'폴드 {idx+1} / {folds.n_splits}','#'*40)
    #Train, Validation data
    X_train, y_train = X[train_idx],y[train_idx]
    X_valid, y_valid = X[valid_idx],y[valid_idx]

    #data set for lgbm
    lgb_train = lgb.Dataset(X_train,y_train)
    lgb_valid = lgb.Dataset(X_valid,y_valid)

    # train lgbm model
    lgb_model = lgb.train(params=lgb_params,
                         train_set = lgb_train,
                         num_boost_round=1000,
                         valid_sets=lgb_valid,
                         feval = gini,                            
                         callbacks=[lgb.early_stopping(stopping_rounds=5)])

    #oof prediction based on test data
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    #based in validation data
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)

    #gini
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'fold {idx+1} gini score : {gini_score}\n')

######################################## 폴드 1 / 5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[24]	valid_0's binary_logloss: 0.155301	valid_0's gini: 0.243445
fold 1 gini score : 0.2434451362266238

######################################## 폴드 2 / 5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1093
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[L

In [23]:
lgb_params = {
    'objective':'binary',
    'learning_rate':0.01,
    'force_row_wise':True,
    'random_state':0,
    'scale_pos_weight': 10,
}

#scale_pos_weight :1.5~10 
for idx, (train_idx,valid_idx) in enumerate(folds.split(X,y)):
    print('#'*40,f'폴드 {idx+1} / {folds.n_splits}','#'*40)
    #Train, Validation data
    X_train, y_train = X[train_idx],y[train_idx]
    X_valid, y_valid = X[valid_idx],y[valid_idx]

    #data set for lgbm
    lgb_train = lgb.Dataset(X_train,y_train)
    lgb_valid = lgb.Dataset(X_valid,y_valid)

    # train lgbm model
    lgb_model = lgb.train(params=lgb_params,
                         train_set = lgb_train,
                         num_boost_round=1000,
                         valid_sets=lgb_valid,
                         feval = gini,                                                   
                         callbacks=[lgb.early_stopping(stopping_rounds=5)])

    #oof prediction based on test data
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    #based in validation data
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)

    #gini
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'fold {idx+1} gini score : {gini_score}\n')

######################################## 폴드 1 / 5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[2]	valid_0's binary_logloss: 0.156172	valid_0's gini: 0.22999
fold 1 gini score : 0.24268400803554507

######################################## 폴드 2 / 5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1093
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[Li

In [24]:
print(eval_gini(y,oof_val_preds))

0.24051405638858542


In [25]:
print(np.sum(np.round(oof_test_preds))/len(oof_test_preds))

0.0


In [26]:
submission['target'] = oof_test_preds
submission.to_csv('base_submission.csv')

In [44]:
from catboost import CatBoostClassifier, Pool

# test_data = Pool(X,y)
model = CatBoostClassifier(iterations =1000, depth=5,learning_rate = 0.01,loss_function='Logloss',verbose=100,od_pval=0.01)

history = model.fit(X,y)
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
print('class : ',preds_class)
print('proba : ',preds_proba)

0:	learn: 0.6779601	total: 159ms	remaining: 2m 38s
100:	learn: 0.1907700	total: 1.99s	remaining: 17.7s
200:	learn: 0.1573888	total: 3.8s	remaining: 15.1s
300:	learn: 0.1533853	total: 5.62s	remaining: 13.1s
400:	learn: 0.1524993	total: 7.45s	remaining: 11.1s
500:	learn: 0.1521244	total: 9.32s	remaining: 9.29s
600:	learn: 0.1518982	total: 11.2s	remaining: 7.44s
700:	learn: 0.1517195	total: 13.1s	remaining: 5.57s
800:	learn: 0.1515691	total: 14.9s	remaining: 3.71s
900:	learn: 0.1514354	total: 16.9s	remaining: 1.85s
999:	learn: 0.1513157	total: 18.7s	remaining: 0us
class :  [0 0 0 ... 0 0 0]
proba :  [[0.97637093 0.02362907]
 [0.97398323 0.02601677]
 [0.97288855 0.02711145]
 ...
 [0.96393678 0.03606322]
 [0.97309088 0.02690912]
 [0.96725185 0.03274815]]


In [28]:
print(preds_proba[:,0])
np.round(preds_proba)
print(np.sum(preds_proba[:,0])/preds_proba.shape[0])
print(np.sum(preds_proba[:,1])/preds_proba.shape[0])

[0.97463692 0.97460532 0.97273679 ... 0.96242168 0.97436631 0.9689999 ]
0.9635820950969949
0.036417904903005484


#lgbm model

In [1]:
def path(name=False):
    if name:
        path = '/kaggle/input/porto-seguro-safe-driver-prediction/'
    else:
        path = 'C:/Users/hakan/Jupyter Notebook/data/porto-seguro-safe-driver-prediction/'
    return path

In [2]:
#import data for jupyter
import pandas as pd

data_path =path()
train = pd.read_csv(data_path+'train.csv',index_col='id')
test = pd.read_csv(data_path+'test.csv',index_col='id')
submission = pd.read_csv(data_path+'sample_submission.csv',index_col='id')

In [3]:
all_data = pd.concat([train,test],ignore_index=True)
all_data = all_data.drop('target',axis=1)
target = train['target']
all_features = all_data.columns

In [4]:
from sklearn.preprocessing import OneHotEncoder

cat_features = [feature for feature in all_features if 'cat' in feature]

onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

#yet not dropped onehot encoded geatures.

In [5]:
print((all_data==-1).sum(axis=1))
all_data['num_missing']=(all_data==-1).sum(axis=1)

0          1
1          2
2          3
3          0
4          2
          ..
1488023    1
1488024    1
1488025    2
1488026    1
1488027    0
Length: 1488028, dtype: int64


In [6]:
all_data.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,num_missing
0,2,2,5,1,0,0,1,0,0,0,...,1,5,8,0,1,1,0,0,1,1
1,1,1,7,0,0,0,0,1,0,0,...,1,1,9,0,1,1,0,1,0,2
2,5,4,9,1,0,0,0,1,0,0,...,2,7,7,0,1,1,0,1,0,3
3,0,1,2,0,0,1,0,0,0,0,...,2,4,9,0,0,0,0,0,0,0
4,0,2,0,1,0,1,0,0,0,0,...,1,1,3,0,0,0,1,1,0,2


In [7]:
ind_features = [feature for feature in all_features if 'ind' in feature]

is_first_feature = True
for ind_feature in ind_features:
    if is_first_feature:
        all_data['mix_ind'] = all_data[ind_feature].astype(str)+'_'
        is_first_feature = False
    else :
        all_data['mix_ind'] += all_data[ind_feature].astype(str)+'_'

In [8]:
all_data.mix_ind

0          2_2_5_1_0_0_1_0_0_0_0_0_0_0_11_0_1_0_
1           1_1_7_0_0_0_0_1_0_0_0_0_0_0_3_0_0_1_
2          5_4_9_1_0_0_0_1_0_0_0_0_0_0_12_1_0_0_
3           0_1_2_0_0_1_0_0_0_0_0_0_0_0_8_1_0_0_
4           0_2_0_1_0_1_0_0_0_0_0_0_0_0_9_1_0_0_
                           ...                  
1488023     0_1_6_0_0_0_1_0_0_0_0_0_0_0_2_0_0_1_
1488024    5_3_5_1_0_0_0_1_0_0_0_0_0_0_11_1_0_0_
1488025     0_1_5_0_0_1_0_0_0_0_0_0_0_0_5_0_0_1_
1488026    6_1_5_1_0_0_0_0_1_0_0_0_0_0_13_1_0_0_
1488027    7_1_4_1_0_0_0_0_1_0_0_0_0_0_12_1_0_0_
Name: mix_ind, Length: 1488028, dtype: object

In [9]:
cat_features+=['mix_ind']
print(cat_features)

['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'mix_ind']


In [10]:
cat_count_features = []

for feature in cat_features:
    val_counts_dict = all_data[feature].value_counts().to_dict()
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x:val_counts_dict[x])

    cat_count_features.append(f'{feature}_count')

In [11]:
all_data['mix_ind'].value_counts()

mix_ind
0_2_1_0_0_1_0_0_0_0_0_0_0_0_7_1_0_0_     2992
0_1_2_0_0_1_0_0_0_0_0_0_0_0_8_1_0_0_     2784
0_1_2_0_0_1_0_0_0_0_0_0_0_0_7_1_0_0_     2568
0_1_1_0_0_1_0_0_0_0_0_0_0_0_7_1_0_0_     2174
0_2_0_0_0_1_0_0_0_0_0_0_0_0_7_1_0_0_     2131
                                         ... 
0_2_0_0_6_0_0_0_1_0_0_0_0_0_4_0_1_0_        1
6_4_7_0_0_0_0_1_0_0_0_0_0_0_7_1_0_0_        1
3_4_8_1_2_0_0_0_1_0_0_0_0_0_12_1_0_0_       1
5_4_6_1_2_0_1_0_0_0_0_0_0_0_4_1_0_0_        1
5_1_8_0_0_0_1_0_0_0_1_0_0_1_3_0_0_1_        1
Name: count, Length: 143769, dtype: int64

In [12]:
drop_features = ['ps_ind_14','ps_ind_10_bin','ps_ind_11_bin','ps_ind_12_bin','ps_ind_13_bin','ps_car_14']

In [13]:
remaining_features = [feature for feature in all_features if ('cat' not in feature and 'calc' not in feature and feature not in drop_features)]

In [14]:
all_data_remaining = all_data[remaining_features + cat_count_features]

In [15]:
all_data_remaining.columns

Index(['ps_ind_01', 'ps_ind_03', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_15',
       'ps_ind_02_cat_count', 'ps_ind_04_cat_count', 'ps_ind_05_cat_count',
       'ps_car_01_cat_count', 'ps_car_02_cat_count', 'ps_car_03_cat_count',
       'ps_car_04_cat_count', 'ps_car_05_cat_count', 'ps_car_06_cat_count',
       'ps_car_07_cat_count', 'ps_car_08_cat_count', 'ps_car_09_cat_count',
       'ps_car_10_cat_count', 'ps_car_11_cat_count', 'mix_ind_count'],
      dtype='object')

In [16]:
from scipy import sparse
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),encoded_cat_matrix],format='csr')

In [17]:
all_data_remaining

,ps_ind_01,ps_ind_03,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,...,ps_car_03_cat_count,ps_car_04_cat_count,ps_car_05_cat_count,ps_car_06_cat_count,ps_car_07_cat_count,ps_car_08_cat_count,ps_car_09_cat_count,ps_car_10_cat_count,ps_car_11_cat_count,mix_ind_count
0,2,5,0,1,0,0,11,0,1,0,...,1028142,1241334,431560,77845,1383070,249663,486510,1475460,18326,6
1,1,7,0,0,1,0,3,0,0,1,...,1028142,1241334,666910,329890,1383070,1238365,883326,1475460,12535,36
2,5,9,0,0,1,0,12,1,0,0,...,1028142,1241334,666910,147714,1383070,1238365,883326,1475460,19943,24
3,0,2,1,0,0,0,8,1,0,0,...,183044,1241334,431560,329890,1383070,1238365,36798,1475460,212989,2784
4,0,0,1,0,0,0,9,1,0,0,...,1028142,1241334,666910,147714,1383070,1238365,883326,1475460,26161,258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488023,0,6,0,1,0,0,2,0,0,1,...,1028142,51211,389558,147714,76138,1238365,486510,1475460,3066,107
1488024,5,5,0,0,1,0,11,1,0,0,...,1028142,1241334,389558,83563,1383070,249663,486510,1475460,23506,26
1488025,0,5,1,0,0,0,5,0,0,1,...,1028142,1241334,666910,329890,1383070,249663,72947,1475460,9725,258
1488026,6,5,0,0,0,1,13,1,0,0,...,1028142,1241334,431560,83563,1383070,1238365,486510,1475460,31344,37


In [18]:
def resumatable(df):
    print(f'DataFrame Shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['Dtype'])
    summary['Missing'] = (df == -1).sum().values
    summary['Unique'] = df.nunique().values
    summary['Dtype'] = None
    for col in df.columns:
        if 'bin' in col or col =='target':
            summary.loc[col,'Dtype'] = 'Binary'
        elif 'cat' in col:
            summary.loc[col,'Dtype'] = 'Categorical'
        elif df[col].dtype == 'float64':
            summary.loc[col,'Dtype'] = 'Continuous'
        elif df[col].dtype == 'int64':
            summary.loc[col,'Dtype'] = 'Ordinal'
    return summary

In [19]:
all_df = resumatable(all_data_remaining)
all_df

DataFrame Shape: (1488028, 32)


,Dtype,Missing,Unique
ps_ind_01,Ordinal,0,8
ps_ind_03,Ordinal,0,12
ps_ind_06_bin,Binary,0,2
ps_ind_07_bin,Binary,0,2
ps_ind_08_bin,Binary,0,2
ps_ind_09_bin,Binary,0,2
ps_ind_15,Ordinal,0,14
ps_ind_16_bin,Binary,0,2
ps_ind_17_bin,Binary,0,2
ps_ind_18_bin,Binary,0,2


In [49]:
num_train = len(train)

X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

In [21]:
len(y) , X.shape

(595212, (595212, 216))

In [22]:
import numpy as np
def eval_gini(y_true, y_pred):
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0]
    L_mid = np.linspace(1/n_samples, 1, n_samples)

    pred_order = y_true[y_pred.argsort()]
    true_order = y_true[y_true.argsort()]
    
    L_pred = np.cumsum(pred_order)/np.sum(true_order)
    G_pred = np.sum(L_mid - L_pred)
    
    
    L_true = np.cumsum(true_order)/np.sum(true_order)
    G_true = np.sum(L_mid - L_true)

    #Normalized Gini coefficient
    return G_pred/G_true

In [23]:
#gini for LGBM
def gini(preds,dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels,preds), True

#optimize hyperparameters

In [24]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2,random_state = 0)
train_bayes = lgb.Dataset(X_train,y_train)
valid_bayes = lgb.Dataset(X_valid,y_valid)

In [36]:
#hyperparameter bounding for bayesian optimizer
param_bounds = {
    'num_leaves':(30,40),
    'lambda_l1':(0.7,0.9),
    'lambda_l2':(0.9,1),
    'feature_fraction':(0.6,0.7),
    'bagging_fraction':(0.6,0.9),
    'min_child_samples':(6,10),
    'min_child_weight':(10,40)
}

#fixed parameters
fixed_params = {    
    'objective':'binary',
    'learning_rate':0.005,
    'bagging_freq':1,
    'force_row_wise':True,
    'random_state':1991,
    'verbose':100
}

In [26]:
def eval_function(num_leaves,lambda_l1,lambda_l2,feature_fraction,bagging_fraction,min_child_samples,min_child_weight):
    params = {'num_leaves':int(round(num_leaves)),
              'lambda_l1':lambda_l1,
              'lambda_l2':lambda_l2,
              'feature_fraction':feature_fraction,
              'bagging_fraction':bagging_fraction,
              'min_child_samples':int(round(min_child_samples)),
              'min_child_weight':min_child_weight,
              'feature_pre_filter':False}
    params.update(fixed_params)

    print('hyperparameter : ',params)

    lgb_model = lgb.train(
        params=params,
        train_set=train_bayes,
        num_boost_round=2500,
        valid_sets=valid_bayes,
        feval = gini,
        callbacks=[lgb.early_stopping(300)]
    )

    preds = lgb_model.predict(X_valid)

    gini_score = eval_gini(y_valid,preds)
    print('gini Score : ',gini_score)

    return gini_score    

In [27]:
from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(f=eval_function,
                                pbounds=param_bounds,
                                random_state=0)

In [28]:
optimizer.maximize(init_points=3,n_iter=6)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
hyperparameter :  {'num_leaves': 34, 'lambda_l1': 0.8205526752143287, 'lambda_l2': 0.9544883182996897, 'feature_fraction': 0.6715189366372419, 'bagging_fraction': 0.7646440511781974, 'min_child_samples': 8, 'min_child_weight': 29.376823391999682, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1547
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 216
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
Training until validation scores don't improve for 300 ro

In [29]:
max_params = optimizer.max['params']

In [30]:
max_params['num_leaves'],max_params['min_child_samples'] = int(round(max_params['num_leaves'])),int(round(max_params['min_child_samples']))

In [31]:
max_params.update(fixed_params)

In [32]:
max_params

{'bagging_fraction': 0.6213108174593661,
 'feature_fraction': 0.608712929970154,
 'lambda_l1': 0.7040436794880651,
 'lambda_l2': 0.9832619845547939,
 'min_child_samples': 9,
 'min_child_weight': 36.10036444740457,
 'num_leaves': 40,
 'objective': 'binary',
 'learning_rate': 0.005,
 'bagging_freq': 1,
 'force_row_wise': True,
 'random_state': 1991}

In [47]:
X, y.sum()/X.shape[0]

(<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 23979130 stored elements and shape (595212, 216)>,
 0.036447517859182946)

In [ ]:
from sklearn.model_selection import StratifiedKFold

#stratified k fold
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

oof_val_preds = np.zeros(X.shape[0])
oof_test_preds = np.zeros(X_test.shape[0])

for idx,(train_idx,valid_idx) in enumerate(folds.split(X,y)):
    print('-'*40,f'fold {idx+1}/{folds.n_splits}','#'*40)

    X_train,y_train = X[train_idx],y[train_idx]
    X_valid,y_valid = X[valid_idx],y[valid_idx]    
    
    dtrain = lgb.Dataset(X_train,y_train)
    dvalid = lgb.Dataset(X_valid,y_valid)

    lgb_model = lgb.train(params = max_params,
                train_set = dtrain,
                num_boost_round=2500,
                valid_sets= dvalid,
                feval=gini,
                callbacks=[lgb.early_stopping(300)])
    
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits

    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)
    
    
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'fold {idx+1} gini score : {gini_score}\n')

---------------------------------------- fold 1/5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1546
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 215
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[1978]	valid_0's binary_logloss: 0.151358	valid_0's gini: 0.298659
fold 1 gini score : 0.2986594054238097

---------------------------------------- fold 2/5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1552
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 215
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.27476

In [ ]:
print('oof gini score : ', eval_gini(y,oof_val_preds))

In [ ]:
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')